# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [64]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from collections import Counter
from nltk import ngrams
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib as mpl
import spacy
from spacy import displacy
from newsapi import NewsApiClient
from path import Path
import csv
nltk.download('vader_lexicon')
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = [20.0, 10.0]
%matplotlib inline



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sameerlakhe/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
analyzer = SentimentIntensityAnalyzer()

In [4]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [80]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

# Function to create a dataframe for english news and french news
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

#this function writes the event data to a csv file to avoid calling the API multiple times
# for the same date range
#def write_csv(data, file_name):
   # with open(file_name, mode='w', encoding='utf-8', newline='\n') as csv_file:
       # writer = csv.DictWriter(csv_file, fieldnames = data[0].keys())

        #writer.writeheader()
        #for event in data:
            #writer.writerow(event)
            
def write_csv(df,file_name):
    file_path = Path(file_name)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

def get_news(query,language,file_name):
    complete_file_name = "./Resources/"+file_name
    path = Path(complete_file_name)
    if path.exists():
        print(f'The file {complete_file_name} exists')
        news_df = pd.read_csv(complete_file_name)
    else:
        print(f'retreiving data from new API for query: {query}, and language:{language}')
        results = newsapi.get_everything(
                    q=query,
                    language=language
                )
        print(f'total results for query "{query}" in langyage:{language} are : {results["totalResults"]}')
        #convert the news articles to dataframe
        news_df = create_df(results["articles"], language)
        write_csv(news_df,complete_file_name)
    return news_df
    

In [79]:
bitcoin_news_en_df = get_news("bitcoin and 2022","en","Bitcoin2022_en.csv")
bitcoin_news_en_df.head()

bitcoin_news_fr_df = get_news("bitcoin and 2022","fr","Bitcoin2022_fr.csv")
bitcoin_news_fr_df.head()

The file ./Resources/Bitcoin2022_en.csv exists
retreiving data from new API for query: bitcoin and 2022, and language:fr
total results for querybitcoin and 2022: 17


In [81]:
# Create the Bitcoin sentiment scores DataFrame
# Create a DataFrame with the news in English
#bitcoin_news_en_df = create_df(bitcoin_news_en["articles"], "en")
# Create a DataFrame with the news in French
#bitcoin_news_fr_df = create_df(bitcoin_news_fr["articles"], "fr")
#concat to get dataframe with news in english and french
bitcoin_news_all_df =pd.concat([bitcoin_news_en_df, bitcoin_news_fr_df])  
bitcoin_news_all_df.head()
# Save to CSV, read the CSV to get the data as there is limit for calling news API multiple times in day
#file_path = Path("./Resources/bitcoin_news_en_fr.csv")
#bitcoin_news_all_df.to_csv(file_path, index=False, encoding='utf-8-sig')

,title,description,text,date,language
0,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en
1,What is Blockchain: Everything You Need to Kno...,<table><tr><td></td></tr><tr><td></td></tr>\r\...,"If you want to pay online, you need to registe...",2022-04-18,en
2,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en
3,Factbox: U.S. companies and their cryptocurren...,A meltdown in one of the world's largest stabl...,May 12 (Reuters) - A meltdown in one of the wo...,2022-05-12,en
4,The Top 12 Blockchain Speakers for 2022,What’s happening in the blockchain industry th...,Whats happening in the blockchain industry thi...,2022-04-26,en


In [39]:
# Fetch the Ethereum news articles
ethereum_news_en_df = get_news("ethereum and 2022","en","Ethereum2022_en.csv")
# Show the total number of news
ethereum_news_en_df.head()

ethereum_news_fr_df = get_news("ethereum and 2022","fr","Ethereum2022_fr.csv")
# Show the total number of news
print(ethereum_news_fr["totalResults"])

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

,title,description,text,date,language
0,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en
1,What is Blockchain: Everything You Need to Kno...,<table><tr><td></td></tr><tr><td></td></tr>\r\...,"If you want to pay online, you need to registe...",2022-04-18,en
2,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en
3,Factbox: U.S. companies and their cryptocurren...,A meltdown in one of the world's largest stabl...,May 12 (Reuters) - A meltdown in one of the wo...,2022-05-12,en
4,The Top 12 Blockchain Speakers for 2022,What’s happening in the blockchain industry th...,Whats happening in the blockchain industry thi...,2022-04-26,en


In [50]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries


# Get sentiment for the text and the title
def get_sentiment_score(news_df):
    title_sent = {
        "title_compound": [],
        "title_pos": [],
        "title_neu": [],
        "title_neg": [],
        "title_sent": [],
    }
    text_sent = {
        "text_compound": [],
        "text_pos": [],
        "text_neu": [],
        "text_neg": [],
        "text_sent": [],
    }
    for index, row in news_df.iterrows():
        try:
            # Sentiment scoring with VADER
            title_sentiment = analyzer.polarity_scores(row["title"])
            title_sent["title_compound"].append(title_sentiment["compound"])
            title_sent["title_pos"].append(title_sentiment["pos"])
            title_sent["title_neu"].append(title_sentiment["neu"])
            title_sent["title_neg"].append(title_sentiment["neg"])
            title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

            text_sentiment = analyzer.polarity_scores(row["text"])
            text_sent["text_compound"].append(text_sentiment["compound"])
            text_sent["text_pos"].append(text_sentiment["pos"])
            text_sent["text_neu"].append(text_sentiment["neu"])
            text_sent["text_neg"].append(text_sentiment["neg"])
            text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
        except AttributeError:
            pass
        
    return title_sent,text_sent

title_sent,text_sent = get_sentiment_score(bitcoin_news_all_df)
    
# Attaching sentiment columns to the News DataFrame

bitcoin_title_sentiment_df = pd.DataFrame(title_sent)
bitcoin_text_sentiment_df = pd.DataFrame(text_sent)

#bitcoin_title_sentiment_df
#bitcoin_text_sentiment_df
bitcoin_news_all_df

#print(title_sent)
#bitcoin_news_all_df = bitcoin_news_all_df.join(bitcoin_title_sentiment_df).join(bitcoin_text_sentiment_df)

#bitcoin_news_all_df.head()


,title,description,text,date,language,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
0,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en,0.0,0.0,1.0,0.0,0.0,-0.0516,0.056,0.882,0.061,-1.0
0,"L’âge d’or du cinéma n’est plus, place à l’âge...",L’une des plus grandes chaînes de salles de ci...,"AMC peut remercier les boursicoteurs, qui lui ...",2022-04-19,fr,0.0,0.0,1.0,0.0,0.0,-0.0516,0.056,0.882,0.061,-1.0
1,What is Blockchain: Everything You Need to Kno...,<table><tr><td></td></tr><tr><td></td></tr>\r\...,"If you want to pay online, you need to registe...",2022-04-18,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Dans le plus grand secret, Snowden a créé une ...",Réfugié en Russie depuis qu'il a reconnu être ...,"Edward Snowden, ancien employé de la CIA, sort...",2022-04-28,fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Le bitcoin perd ses gains de 2022 et passe sou...,"Londres - Le bitcoin, la principale cryptomonn...","A 29.764 dollars, son plus bas mardi, le bitco...",2022-05-10,fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Factbox: U.S. companies and their cryptocurren...,A meltdown in one of the world's largest stabl...,May 12 (Reuters) - A meltdown in one of the wo...,2022-05-12,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Envoyer des cryptos sur Telegram, c’est possible",Après l’abandon du projet de crypto de Telegra...,"À linstar de Meta, qui est le propriétaire de ...",2022-05-02,fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Top 12 Blockchain Speakers for 2022,What’s happening in the blockchain industry th...,Whats happening in the blockchain industry thi...,2022-04-26,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Le bitcoin perd ses gains de 2022 et passe sou...,"Le bitcoin, la principale cryptomonnaie, a vu ...","Le bitcoin, la principale cryptomonnaie, a vu ...",2022-05-10,fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create the Ethereum sentiment scores DataFrame
# YOUR CODE HERE!

In [8]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!

In [9]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---